# Type of examples for which ner blinding predict the wrong relation vs the baseline which predicts the correct relations

In [1]:
%load_ext autoreload

In [2]:
%autoreload
import os
from sys import path
import re
import pandas as pd
path.append('../../..')
import numpy as np
from relation_extraction.data.converters.converter_i2b2 import relation_dict
output_path = '/scratch/geeticka/relation-extraction-result/i2b2-analyze/'
def res(path): return os.path.join(output_path, path)
original_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/i2b2/pre-processed/original/test_original.txt')
ner_blinding_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/i2b2/pre-processed/ner_blinding/test_ner_blinding.txt')

In [3]:
relation_dict

{0: 'TrIP',
 1: 'TrWP',
 2: 'TrCP',
 3: 'TrAP',
 4: 'TrNAP',
 5: 'TeRP',
 6: 'TeCP',
 7: 'PIP',
 8: 'None'}

In [4]:
def read_answers_line(line):
    linenum, relation = line.strip().split()
    return linenum, relation

In [5]:
def asstring(list_of_strings):
    return " ".join(list_of_strings)

In [6]:
def read_sentence_and_entities(line):
    line = line.strip().split()
    sentence = line[5:]
    relation = relation_dict[int(line[0])]
    entity1_idx = (int(line[1]), int(line[2]))
    entity2_idx = (int(line[3]), int(line[4]))
    entity1 = sentence[entity1_idx[0] : entity1_idx[1] + 1]
    entity2 = sentence[entity2_idx[0] : entity2_idx[1] + 1]
    
    return relation, asstring(entity1), asstring(entity2), asstring(sentence)

In [7]:
needed_linenum_and_relation = {}

In [8]:
with open(res('answers_for_dev-ner-blinding.txt')) as textfile1, open(res("answers_for_dev-baseline.txt")) as textfile2, \
open(res('answers_for_dev_gold-baseline.txt')) as textfile3: 
    for x, y, z in zip(textfile1, textfile2, textfile3):
        linenum, ner_blinding_relation = read_answers_line(x)
        _, baseline_relation = read_answers_line(y)
        _, gold_relation = read_answers_line(z)
        if baseline_relation == gold_relation and ner_blinding_relation != gold_relation:
            needed_linenum_and_relation[int(linenum) - 1] = (baseline_relation, ner_blinding_relation)

In [9]:
len(list(needed_linenum_and_relation.keys()))

2024

In [10]:
len(list(needed_linenum_and_relation.keys()))/len(open(res('answers_for_dev_gold-baseline.txt')).readlines()) * 100

10.589096996965575

Note that we list baseline first and then the ner blinded version but the prediction of the baseline version is correct (gold) whereas the ner blinded is incorrect. 

In [11]:
entities_blinded_per_sentence = []
trip = 0; trwp = 0; trcp = 0; trap = 0; trnap = 0; terp = 0; tecp = 0; pip = 0; none = 0

In [12]:
print('We print the baseline first and then the ner blinded version. Gold relation corresponds to baseline\n\n')
curr_linenum = 0
with open(original_sentences_path) as original_sentences, open(ner_blinding_sentences_path) as ner_blinding_sentences:
    for x, y in zip(original_sentences, ner_blinding_sentences):
        needed_linenums = list(needed_linenum_and_relation.keys())
        if curr_linenum in needed_linenums:
            _, e1_b, e2_b, s_b = read_sentence_and_entities(x.strip())
            _, e1_c, e2_c, s_c = read_sentence_and_entities(y.strip())
            r_b, r_c = needed_linenum_and_relation[curr_linenum]
            entities_blinded_per_sentence.append(s_c.split().count('ENTITY'))
            if r_b == 'TrIP': trip += 1
            elif r_b == 'TrWP': trwp += 1
            elif r_b == 'TrCP': trcp += 1
            elif r_b == 'TrAP': trap += 1
            elif r_b == 'TrNAP': trnap += 1
            elif r_b == 'TeRP': terp += 1
            elif r_b == 'TeCP': tecp += 1
            elif r_b == 'PIP': pip += 1
            elif r_b == 'None': none += 1
            print('Predicted Relation: \t {0}, {1} \nEntities: \t {2}, {3} \t {4}, {5} \nSentences: \n\t{6} \n\t {7}'.format(
                    r_b, r_c, e1_b, e1_c, e2_b, e2_c, s_b, s_c))
            print('\n')
        curr_linenum += 1

We print the baseline first and then the ner blinded version. Gold relation corresponds to baseline


Predicted Relation: 	 TrAP, None 
Entities: 	 a VVI pacer placement, a ENTITY ENTITY ENTITY 	 sick sinus syndrome, ENTITY 
Sentences: 
	2 ) Coronary artery disease , status post coronary artery bypass graft , status post anterior myocardial infarction , sick sinus syndrome , status post a VVI pacer placement . 
	 2 ) ENTITY , ENTITY , ENTITY , ENTITY , ENTITY a ENTITY ENTITY ENTITY


Predicted Relation: 	 TrAP, None 
Entities: 	 closure, ENTITY 	 his patent foramen ovale ( PFO ), ENTITY ( ENTITY ) 
Sentences: 
	He was taken to the operating room where he underwent a mini-thoracotomy with closure of his patent foramen ovale ( PFO ) . 
	 He was taken to the ENTITY where he underwent a ENTITY with ENTITY ( ENTITY )


Predicted Relation: 	 TrAP, PIP 
Entities: 	 a mini-thoracotomy, a ENTITY 	 his patent foramen ovale ( PFO ), ENTITY ( ENTITY ) 
Sentences: 
	He was taken to the operating ro

Predicted Relation: 	 None, TeRP 
Entities: 	 Adenosine stress MIBI, ENTITY ENTITY 	 a 70% lesion in the SVG to PDA, a 70% ENTITY in the ENTITY to ENTITY 
Sentences: 
	Adenosine stress MIBI revealed a small reversible defect in the PDA territory and the patient underwent coronary catheterisation on 1/6/0004 which revealed a 70% lesion in the SVG to PDA -> cypher stented successfully . 
	 ENTITY ENTITY revealed a small reversible ENTITY in the ENTITY ENTITY and the ENTITY underwent ENTITY on 1/6/0004 which revealed a 70% ENTITY in the ENTITY to ENTITY -> ENTITY ENTITY successfully .


Predicted Relation: 	 None, PIP 
Entities: 	 Lasix, ENTITY 	 elevation, ENTITY 
Sentences: 
	baseline CRI ( 1.2-1.6 ) with elevation at admission probably prerenal ; will monitor BUN / Cr after giving Lasix 
	 ENTITY ENTITY ( 1.2-1.6 ) with ENTITY at ENTITY probably ENTITY ; will monitor ENTITY / ENTITY after giving ENTITY


Predicted Relation: 	 None, PIP 
Entities: 	 Cr, ENTITY 	 elevation, ENTITY 
Sente

Predicted Relation: 	 None, PIP 
Entities: 	 needle jejunostomy tube placement, ENTITY ENTITY 	 ventral hernia, ENTITY 
Sentences: 
	On 1/12/92 near total gastrectomy with retrocolic Bill-Roth II gastrojejunostomy , portal node dissection , repair of ventral hernia , and needle jejunostomy tube placement . 
	 On 1/12/92 near total ENTITY with ENTITY ENTITY , ENTITY , ENTITY of ENTITY , and ENTITY ENTITY


Predicted Relation: 	 None, PIP 
Entities: 	 near total gastrectomy, near total ENTITY 	 ventral hernia, ENTITY 
Sentences: 
	On 1/12/92 near total gastrectomy with retrocolic Bill-Roth II gastrojejunostomy , portal node dissection , repair of ventral hernia , and needle jejunostomy tube placement . 
	 On 1/12/92 near total ENTITY with ENTITY ENTITY , ENTITY , ENTITY of ENTITY , and ENTITY ENTITY .


Predicted Relation: 	 TrAP, TeRP 
Entities: 	 Suture, ENTITY 	 right temporal head wound, ENTITY 
Sentences: 
	Suture of right temporal head wound , performed on April 14 , 1994 . 
	 ENTI

Predicted Relation: 	 TrAP, None 
Entities: 	 Tylenol, ENTITY 	 pain, ENTITY 
Sentences: 
	Tylenol , 650 mg. PO q.4 hours , p.r.n. pain ; 
	 ENTITY , 650 mg. ENTITY ENTITY , ENTITY


Predicted Relation: 	 TrAP, None 
Entities: 	 Milk of Magnesia, ENTITY of ENTITY 	 constipation, ENTITY 
Sentences: 
	Milk of Magnesia , 30 ml. PO q.day , p.r.n. constipation ; 
	 ENTITY of ENTITY , ENTITY ENTITY , ENTITY


Predicted Relation: 	 TrAP, None 
Entities: 	 chemotherapy, ENTITY 	 the non-small-cell lung cancer, the ENTITY 
Sentences: 
	Asthma , history of atrial fibrillation , non-small-cell lung cancer removed from his right lung by Dr. Vital four years prior to admission , status post chemotherapy and radiation for the non-small-cell lung cancer , gastroesophageal reflux disease , and history of constipation . 
	 ENTITY , ENTITY of ENTITY , ENTITY ENTITY from his ENTITY by ENTITY prior to ENTITY , status ENTITY and ENTITY for the ENTITY , ENTITY , and ENTITY .


Predicted Relation: 	 TeRP, No



Predicted Relation: 	 None, PIP 
Entities: 	 the patient 's human immunodeficiency virus test, the ENTITY ENTITY 	 connective tissue disorder stigmata, ENTITY ENTITY 
Sentences: 
	Collagen vascular disease could not be definitively ruled out ; however , given the patient 's lack of connective tissue disorder stigmata it was thought to be less likely , and the patient 's human immunodeficiency virus test was negative . 
	 ENTITY could not be definitively ruled out ; however , given the ENTITY ENTITY of ENTITY ENTITY it was thought to be less likely , and the ENTITY ENTITY was ENTITY .


Predicted Relation: 	 None, PIP 
Entities: 	 Grave's, ENTITY 	 hypothyroidism, ENTITY 
Sentences: 
	2. Grave's , status post radiation resulting in hypothyroidism . 
	 2. ENTITY , ENTITY ENTITY resulting in ENTITY


Predicted Relation: 	 None, TrAP 
Entities: 	 herbal medication, ENTITY 	 human immunodeficiency virus, ENTITY 
Sentences: 
	In addition , consideration was given to sarcoidosis , human imm

Predicted Relation: 	 TeRP, None 
Entities: 	 CT PELVIS, ENTITY ENTITY 	 An abnormal loop, An ENTITY loop 
Sentences: 
	CT PELVIS ( 07-17 ) An abnormal loop of slightly dilated small bowel with fecalization and possible bowel wall thickening . 
	 ENTITY ENTITY ( 07-17 ) An ENTITY loop of slightly ENTITY ENTITY with ENTITY and possible ENTITY .


Predicted Relation: 	 TeRP, None 
Entities: 	 CT PELVIS, ENTITY ENTITY 	 fecalization, ENTITY 
Sentences: 
	CT PELVIS ( 07-17 ) An abnormal loop of slightly dilated small bowel with fecalization and possible bowel wall thickening . 
	 ENTITY ENTITY ( 07-17 ) An ENTITY loop of slightly ENTITY ENTITY with ENTITY and possible ENTITY .


Predicted Relation: 	 TeRP, None 
Entities: 	 CT PELVIS, ENTITY ENTITY 	 bowel wall thickening, ENTITY 
Sentences: 
	CT PELVIS ( 07-17 ) An abnormal loop of slightly dilated small bowel with fecalization and possible bowel wall thickening . 
	 ENTITY ENTITY ( 07-17 ) An ENTITY loop of slightly ENTITY ENTITY with EN

Predicted Relation: 	 TeCP, TrAP 
Entities: 	 evaluation, ENTITY 	 his purulent sputum, his ENTITY 
Sentences: 
	Infectious Disease consultation was obtained for evaluation of his purulent sputum and the patient was covered with Vancomycin , Ceftazidime and Gentamicin . 
	 ENTITY was obtained for ENTITY of his ENTITY and the ENTITY was covered with ENTITY , ENTITY and ENTITY .


Predicted Relation: 	 TrAP, TeRP 
Entities: 	 thoracotomy instruments, ENTITY ENTITY 	 tamponade, ENTITY 
Sentences: 
	The patient &apos;s prothrombin time had been 23 early the same morning and tamponade was considered and thoracotomy instruments obtained . 
	 The ENTITY ENTITY had been 23 ENTITY the same ENTITY and ENTITY was considered and ENTITY ENTITY obtained .


Predicted Relation: 	 None, TrIP 
Entities: 	 respiratory rates, ENTITY 	 acutely hypoxic, ENTITY 
Sentences: 
	Very early in the morning on postoperative day # 3 , the patient became acutely hypoxic after several hours of marginal oxygenation wi



Predicted Relation: 	 TrIP, PIP 
Entities: 	 NTG, ENTITY 	 the patients pain, the ENTITY pain 
Sentences: 
	In the Amanda the patients pain resolved with NTG and morphine . 
	 In the ENTITY the ENTITY pain resolved with ENTITY and ENTITY .


Predicted Relation: 	 PIP, None 
Entities: 	 Ischemia, ENTITY 	 Chest Pain, ENTITY 
Sentences: 
	# Chest Pain / Ischemia - Patient with history of CAD a/p MI . 
	 # ENTITY / ENTITY - ENTITY with ENTITY of ENTITY ENTITY .


Predicted Relation: 	 TeRP, TrAP 
Entities: 	 CTA, ENTITY 	 PE, ENTITY 
Sentences: 
	CTA neg for PE . 
	 ENTITY ENTITY for ENTITY


Predicted Relation: 	 PIP, None 
Entities: 	 swelling, ENTITY 	 DVT, ENTITY 
Sentences: 
	He has no JVD , no edema in legs ( other than swelling from DVT ) . 
	 He has no ENTITY , no ENTITY in ENTITY ( other than ENTITY from ENTITY ) .


Predicted Relation: 	 PIP, None 
Entities: 	 MI, ENTITY 	 Chest Pain, ENTITY 
Sentences: 
	# Chest Pain / Ischemia - Patient with history of CAD a/p MI . 
	 # ENTI

In [13]:
print(trip, trwp, trcp, trap, trnap, terp, tecp, pip, none)

34 0 54 378 6 367 73 294 818


In [14]:
trip + trwp + trcp + trap + trnap + terp + tecp + pip + none

2024

In [15]:
(np.mean(entities_blinded_per_sentence), np.max(entities_blinded_per_sentence), np.min(entities_blinded_per_sentence),
np.std(entities_blinded_per_sentence), np.median(entities_blinded_per_sentence))

(8.390316205533598, 48, 1, 5.818383118483416, 7.0)

In [16]:
num_sentences_where_entities_blinded = 0 # count the number of sentences where the numbers were blinded
for numbers in entities_blinded_per_sentence:
    if numbers > 0:
        num_sentences_where_entities_blinded += 1

In [17]:
num_sentences_where_entities_blinded

2024